<a href="https://colab.research.google.com/github/vij37vibhor/queryWeb/blob/main/WebBasedQ%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing Dependencies




In [ ]:
!apt-get install -y wkhtmltopdf
!pip install fastapi uvicorn pyngrok pdfkit requests beautifulsoup4 PyPDF2 llama-index python-dotenv
!pip install llama-index llama-index-core transformers llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install -U bitsandbytes accelerate transformers
!pip install python-multipart
!pip install fastapi uvicorn python-dotenv pyngrok nest_asyncio

# Importing Dependencies
Dependencies from Core Llama Index (LI), LI LLM, embeddings, transformers and torch  

In [2]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.settings import Settings

# LLM Configuration
This snippet contains the LLM model, its configuration that we have used. You can change the model to be used by updating it in *Line#1 - MODEL_NAME*, for embedding used, change *model_name* in Line#26.

**NOTE**: If you are running into  memory constrains while executing the file, it is recommended to run on colab with a T4 runtime.

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,
    model_name=MODEL_NAME,
    tokenizer=tokenizer,
    model=model,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.embed_model = embed_model

# Index.html
Here we are writing the index.html file that will serve as the landing page for our app, executing the cell will load and reload the file in colab env. This contains the styling of our form displayed on the app.

In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Query from Webpage</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: url('...') no-repeat center center fixed;
            background-size: 45%;
            background-position: center;
            background-repeat: no-repeat;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .overlay {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.6); /* Dark overlay for readability */
            z-index: -1;
        }
        .container {
            background: rgba(255, 255, 255, 0.85); /* Semi-transparent white */
            padding: 20px 40px 20px 20px;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.2);
            max-width: 500px;
            width: 100%;
            text-align: center;
        }
        input, textarea {
            width: 100%;
            padding: 8px;
            margin-top: 5px;
            margin-bottom: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            background: #007BFF;
            color: white;
            border: none;
            padding: 10px;
            width: 100%;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background: #0056b3;
        }
        #response {
            margin-top: 15px;
            padding: 10px;
            border-radius: 5px;
            background: #e9ecef;
            text-align: left;
            white-space: pre-wrap;
        }
        .labels {
            text-align: left;
            display: block;
            font-weight: bold;
            margin-bottom: 5px;
        }
    </style>
    <script>
        async function submitForm(event) {
            event.preventDefault();

            let formData = new FormData(event.target);
            let urls = formData.get("urls");
            let query = formData.get("query");

            let responseDiv = document.getElementById("response");
            responseDiv.innerHTML = "Processing...";

            try {
                let response = await fetch("/query_documents/", {
                    method: "POST",
                    headers: { "Content-Type": "application/x-www-form-urlencoded" },
                    body: `urls=${encodeURIComponent(urls)}&query=${encodeURIComponent(query)}`
                });

                let result = await response.json();
                responseDiv.innerHTML = `<strong>Response:</strong><br>${JSON.stringify(result, null, 2)}`;
            } catch (error) {
                responseDiv.innerHTML = "Error processing request.";
            }
        }
    </script>
</head>
<body>
    <div class="overlay"></div>
    <div class="container">
        <h2>QueryWeb</h2>
        <form onsubmit="submitForm(event)">
            <label for="urls" class="labels">Enter URLs (comma-separated):</label>
            <input type="text" id="urls" name="urls" required>

            <label for="query" class="labels">Enter Query:</label>
            <textarea id="query" name="query" rows="4" required></textarea>

            <button type="submit">Query</button>
        </form>

        <div id="response"></div>
    </div>
</body>
</html>


# Web Scraping and RAG
Importing dependencies for web scraping, fast API, and other libraries. **Ensure before running this cell you have .env file loaded with correct NGROK_AUTHTOKEN**. This snippet contains the main logic for retrieving information from webpage, given the URL/s and saving them in the runtime's storage in folder my_pdfs (Since we are saving them as PDFs to be consumed later.) The fast API has the operation *query_documents* which is executed on submission of our form, which queries the input to our llm and then displays the result.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pdfkit
from PyPDF2 import PdfMerger
import glob
from fastapi import FastAPI, HTTPException, Form
from fastapi.responses import HTMLResponse, FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import List
import os
import time
import threading
from pyngrok import ngrok
from dotenv import load_dotenv
import nest_asyncio
import uvicorn

nest_asyncio.apply()
load_dotenv()
ngrok_token = os.getenv("NGROK_AUTHTOKEN")
ngrok.set_auth_token(ngrok_token)


save_folder = "/content/my_pdfs/"
os.makedirs(save_folder, exist_ok=True)

pdfkit_config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

class URLQueryInput(BaseModel):
    urls: List[str]
    query: str

def fetch_webpages(urls):
    headers = {"User-Agent": "Mozilla/5.0"}
    responses = {}

    for url in urls:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                responses[url] = response.text
            else:
                print(f"Failed to fetch {url}, Status Code:", response.status_code)
                responses[url] = None
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            responses[url] = None

    return responses

def save_webpages_as_pdfs(urls):
    webpages = fetch_webpages(urls)
    pdfkit_config = pdfkit.configuration()

    for url, html_content in webpages.items():
        if html_content:
            soup = BeautifulSoup(html_content, "html.parser")
            main_content = soup.find("div", {"id": "content"}) or soup.get_text()
            filename = os.path.join(save_folder, f"{url.replace('https://', '').replace('http://', '').replace('/', '_')}.pdf")
            pdfkit.from_string(str(main_content), filename, configuration=pdfkit_config)
            print(f"Saved: {filename}")
        else:
            print(f"No content extracted for {url}.")
app = FastAPI()
@app.get("/", response_class=HTMLResponse)
def serve_form():
    index_path = "index.html"

    if not os.path.exists(index_path):
        return HTMLResponse(content="<h1>Error: index.html not found!</h1>", status_code=404)
    try:
        with open(index_path, "r", encoding="utf-8") as file:
            html_content = file.read()
            return HTMLResponse(content=html_content)
    except Exception as e:
        return HTMLResponse(content=f"<h1>Error reading index.html: {str(e)}</h1>", status_code=500)


@app.post("/query_documents/")
def query_documents(urls: str = Form(...), query: str = Form(...)):
    try:
        url_list = urls.split(",")
        save_webpages_as_pdfs(url_list)

        pdf_files = glob.glob(os.path.join(save_folder, "*.pdf"))
        print("PDF Files Found:", pdf_files)

        documents = SimpleDirectoryReader(input_dir="/content/my_pdfs").load_data()

        index = VectorStoreIndex.from_documents(documents)
        query_engine = index.as_query_engine()
        updated_query = str(query) + 'Ensure answers are only from the sample website being ingested. If you do not have the answer, respond with the following statement encased in triple-double quotes only """I could not find the response based on webpage data ingested""". No need to add the context info or any details. Remove any special characters for negative responses'
        response = query_engine.query(updated_query)
        print(response)

        #Removing the scraped indices before ending execution
        for file in glob.glob("/content/my_pdfs/*.pdf"):
          os.remove(file)
          print(f"Deleted: {file}")

        return response.response.encode('ascii', 'ignore').decode()

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)


fastapi_thread = threading.Thread(target=run_fastapi)
fastapi_thread.start()

time.sleep(5)

public_url = ngrok.connect(8000).public_url
print(f"FastAPI is running at: {public_url}")


In [7]:
!pkill -f ngrok    #Use only to kill running app, preferably start the runtime again!